# 02 Regime Long-Short Demo
Risk-on vs risk-off macro regimes and a biotech/pharma long-short example with risk-balanced sizing.

- **Risk-on**: lower rates, rising equities, calmer VIX.
- **Risk-off**: stress or uncertainty.
- **Long-short**: long biotech vs short pharma when both macro and spread momentum agree.
- **Risk-balance**: position sizes scaled by XBI/XPH vol so each leg contributes similar risk.

In [ ]:
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

repo_root = Path.cwd().resolve()
if not (repo_root / 'src').exists():
    repo_root = repo_root.parent
if str(repo_root) not in sys.path:
    sys.path.append(str(repo_root))

from src.data.etf_loader import load_clean_prices
from src.data.macro_loader import load_tnx_10y, load_vix
from src.signals.regime import compute_monthly_features, classify_regime
from src.signals.ls_biotech_pharma import compute_spread_momentum, build_monthly_ls_weights
from src.portfolio.vol_target import estimate_rolling_vol
from src.backtest.engine import run_backtest
from src.analysis.metrics import compute_cagr, compute_annual_vol, compute_sharpe, compute_max_drawdown

%matplotlib inline


In [ ]:
prices = load_clean_prices()[['XBI','XPH','SPY']].dropna()
prices.head()


### Macro inputs
TNX (10Y) and VIX pulled from Yahoo; resampled to month-end for regime features.

In [ ]:
tnx_yield = load_tnx_10y(start=prices.index.min().strftime('%Y-%m-%d'), end=prices.index.max().strftime('%Y-%m-%d')).reindex(prices.index).ffill()
vix = load_vix(start=prices.index.min().strftime('%Y-%m-%d'), end=prices.index.max().strftime('%Y-%m-%d')).reindex(prices.index).ffill()
spy_prices = prices['SPY']
monthly_features = compute_monthly_features(tnx_yield, spy_prices, vix)
regime_labels = classify_regime(monthly_features)
print(f"Risk-on fraction: {regime_labels.mean():.1%}")
regime_labels.tail()


### Spread momentum and volatility
Use 6M log spread momentum as a confirmation filter; vol used for risk-balancing the legs.

In [ ]:
spread_mom = compute_spread_momentum(prices[['XBI','XPH']], lookback_months=6)
daily_returns = prices[['XBI','XPH']].pct_change().fillna(0.0)
vol_df = estimate_rolling_vol(daily_returns)
spread_mom.tail()


### Build risk-balanced daily weights
Requires macro risk-on *and* positive spread momentum.

In [ ]:
ls_weights_daily = build_monthly_ls_weights(
    regime_labels=regime_labels,
    prices=prices[['XBI','XPH']],
    vol_df=vol_df,
    spread_momentum=spread_mom,
    target_gross_exposure=1.0,
    spread_mom_threshold=0.0,
)
ls_weights_daily.head()


In [ ]:
bt_result = run_backtest(prices[['XBI','XPH']], ls_weights_daily, transaction_cost_bps=10)
cagr = compute_cagr(bt_result.daily_returns)
ann_vol = compute_annual_vol(bt_result.daily_returns)
sharpe = compute_sharpe(bt_result.daily_returns)
max_dd = compute_max_drawdown(bt_result.equity_curve)
print(f'CAGR:   {cagr:.2%}')
print(f'Vol:    {ann_vol:.2%}')
print(f'Sharpe: {sharpe:.2f}')
print(f'Max DD: {max_dd:.2%}')


### Plots
Equity curve vs XLV and the XBI/XPH ratio; regime/spread flags shown for context.

In [ ]:
xbi_xph_ratio = prices['XBI'] / prices['XPH']
xlv = load_clean_prices()['XLV'].reindex(prices.index).dropna()
fig, ax = plt.subplots(figsize=(12, 6))
bt_result.equity_curve.plot(ax=ax, label='LS Equity')
(1 + xbi_xph_ratio.pct_change().fillna(0)).cumprod().plot(ax=ax, label='XBI/XPH ratio')
(1 + xlv.pct_change().fillna(0)).cumprod().plot(ax=ax, label='XLV')
ax.set_title('Biotech vs Pharma LS vs Benchmarks')
ax.set_ylabel('Cumulative wealth (normalized)')
ax.legend()
plt.tight_layout()
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(12, 3))
regime_daily = regime_labels.reindex(prices.index, method='ffill').fillna(0)
spread_sign = (spread_mom > 0).reindex(regime_daily.index, method='ffill').fillna(0)
(regime_daily * spread_sign).plot(ax=ax, label='Active (regime & spread)', color='green')
ax.set_title('Regime & Spread Momentum Flags (1=active)')
ax.set_ylim(-0.1, 1.1)
plt.tight_layout()
plt.show()


### Rationale
- **Risk-balanced legs:** scale notional by inverse vol so each side contributes similar risk.
- **Dual gating:** require macro risk-on *and* positive spread momentum to avoid fighting a downtrend.
- **Risk-off:** flat when filters are not satisfied.